In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import nrrd
import k3d
import sys
import torch
sys.path.append('..')
from data_utils import AsocaDataModule
from data_utils.helpers import get_volume_pred, patches2vol

In [ ]:
adm = AsocaDataModule(patch_size=32, stride=22, output_dir='../dataset')

In [ ]:
bs = 500

In [ ]:
vdl, meta = adm.volume_dataloader(2, batch_size=bs)

In [ ]:
meta

In [ ]:
patches = torch.empty((meta['n_patches'],*meta['shape_patched'][3:]))

In [ ]:
cur = 0
for x, _ in vdl:
    patches[cur:cur+bs] = x.squeeze(1)
    cur += bs

In [ ]:
volume = get_volume_pred(patches, meta, 22).numpy()

In [ ]:
ds = h5py.File('../dataset/asoca.hdf5', 'r')

In [ ]:
plt.imshow(ds['train']['volumes'][0][0])

In [ ]:
_, axes = plt.subplots(8,8, figsize=(20,20))
for i in range(8):
    for j in range(8):
        axes[i][j].imshow(ds['train']['volumes'][12][i*j+j])

In [ ]:
import k3d
%matplotlib notebook

In [ ]:
k = 12
volume = ds['train']['volumes'][k]
volume_mask = ds['train']['masks'][k]

In [ ]:
k3d_volume = k3d.volume(
    volume[::4,::4,::4].astype(np.float32),
    alpha_coef=10,
    shadow='dynamic',
    samples=600,
    shadow_res=128,
    shadow_delay=50,
#     color_range=[150,750],
    color_map=(np.array(k3d.colormaps.matplotlib_color_maps.Gist_heat).reshape(-1,4)
               * np.array([1,1.75,1.75,1.75])).astype(np.float32),
    compression_level=9
)
# size = volume.shape
# k3d_volume.transform.bounds = [-size[0]/2,size[0]/2,
#                            -size[1]/2,size[1]/2,
#                            -size[2]/2,size[2]/2]

plot = k3d.plot(camera_auto_fit=True)
plot += k3d_volume

# k3d_volume_mask = k3d.volume(
#     volume_mask.astype(np.float32),
#     alpha_coef=1000,
#     samples=600,
#     color_range=[0],
#     compression_level=9
# )
# size = volume_mask.shape
# k3d_volume_mask.transform.bounds = [-size[0]/2,size[0]/2,
#                            -size[1]/2,size[1]/2,
#                            -size[2]/2,size[2]/2]

# plot += k3d_volume_mask

plot.lighting = 2
plot.display()

In [ ]:
data, header = nrrd.read('../dataset/Train/1.nrrd', index_order='C')
data_mask, _ = nrrd.read('../dataset/Train_Masks/1.nrrd', index_order='C')

In [ ]:
mean = data.mean()
std = data.std()

In [ ]:
data_n = (data - mean) / std

In [ ]:
_, axes = plt.subplots(2,2, figsize=(20,10))
axes[0,0].hist(data.flatten(), bins=100, log=True)
axes[0,1].hist(data_n.flatten(), bins=100, log=True)
axes[1,0].imshow(data[100])
axes[1,1].imshow(data_n[100])
plt.show()

In [ ]:
data[data<0] = -3000
data[data>350] = -3000

In [ ]:
plt.hist(data[data!=-3000].flatten(), bins=100)
plt.show()

In [ ]:
plot = k3d.plot(camera_auto_fit=True, fps=60)

# plot += k3d.points(lattice, point_size=.05)
i = 0
plot += k3d.sparse_voxels(
    sparse_d.astype(np.uint16),
    (183,512,512),
    name='input',
    alpha_coef=100,
    samples=600,
    color_range=[0, 350],
    color_map=k3d.colormaps.matplotlib_color_maps.Coolwarm,
)

plot += k3d.volume(
    data_mask.astype(np.float32),
    name='target',
    alpha_coef=100,
    samples=600,
    color_range=[0, 1],
    color_map=k3d.colormaps.matplotlib_color_maps.Oranges,
)
plot.display()

In [ ]:
def get_sparse_volume(volume):
    assert len(volume.shape) == 3, f'Volume not 3D, shape: {volume.shape}'
    W, H, D = volume.shape
    X = np.linspace(-0.5, 0.5, W)
    Y = np.linspace(-0.5, 0.5, H)
    Z = np.linspace(-0.5, 0.5, D)


    X, Y, Z = np.meshgrid(X,Y,Z, indexing='ij')
    grid = np.array([X,Y,Z])
    grid = grid.transpose((1,2,3,0))
    volume_sparse = np.concatenate([grid, volume[...,None]], axis=-1)
    volume_sparse = volume_sparse.reshape(-1,4)
  
    return volume_sparse

In [ ]:
d = data.copy()

In [ ]:
d[d==-3000] = 0

In [ ]:
sparse_d = get_sparse_volume(d)

In [ ]:
sparse_d.shape

In [ ]:
sparse_d.min()

In [ ]:
sparse_d = sparse_d + 3024